In [16]:
from PsQ_Solver_CNN import *
import numpy as np

# The Syntax of Sudoku (Grids),  Part II: Solve the puzzle! 

AlexPfaff, June-Aug 2025  
nonintersective@gmail.com 

<br> 

## What's going on here? 

* Generate a dataset of blanked grids = Sudoku puzzles with 1-k blanks. The variable 'k' must be between 1 and 81 (GRIDSIZE), default setting = 64. Since a classical 9 x 9 Sudoku puzzle must have at least 17 given digits (McGuire, Tugemann, and Civario, 2014), it seems a bith unethical to set k > 64, but feel free to do what you must do. 
Input data are created on the basis of a set of valid (complete) Sudoku grids (= labels) where 1-k cells are randomly selected and their values replaced by 0 ($\rightarrow$ the blanks).  
* Train a model on the data: input shape (n, 9, 9) - the blanked grids; output shape (n, 9, 9). So the model predicts a complete grid array (= 9 x 9 = 81 values). Motivation: if only the missing values were to be predicted, we'd need models for 1 blank, 2 blanks ... 64 blanks. Thus always predicting the entire grid is the more economic and elegant option, in spite of the overhead. 
* Strategy: '**Best Argmax wins!**'
   - inserting a full grid (or even only a set of values) in one go is not an appropriate way to solve a Sudoku; we want to solve step by step, one cell at a time
   - predicting 81 values in one go has various pitfalls: 1. the model can easily make a false prediction for a given blank cell (= 0); 2. the model may even make a false prediction for a non-blank cell, i.e. a value that we actually already have
   - instead: predict k times, pick one value, dismiss the rest, insert, repeat (= recursive procedure):  
        (i) for a given grid g: model.predict(g) $\rightarrow$ 81 output values $\rightarrow$ ignore those that have a non-zero digit in the corresponding cell in the input grid   
        (ii) out of the remaining values, pick the one with the highest overall probaility = the highest respective Argmax and insert in g <br>(e.g. the probability P[ cell (2, 3) = 7 ] = 0.9, and P[ cell (5, 4) = 3 ] is 0.95: we pick the latter and insert the value '3' into grid g @position (5,4))    
        (iii) repeat steps (i)-(iii) until a.) g contains no more 0s (success), or b.) the prediction is false (failure).   

<br>
This notebook comtains code to generate the dataset, train a model, and save both. Moreover, it contains a simple solver (class SudokuPlayer) to be used in the notebook, press 'Enter' for every new prediction. Be sure, model and dataset are loaded.  

NB: model 'solver1.keras' and dataset 'reduced_dataset_1.npy' can be found at https://github.com/A-Lex-McLee/PseudoQ-2.1/tree/main; you can simply load those if you do not want to produce a new dataset and train you r own model (which can be rather (run-) time-intensive !!!) 

For more information & discussion, see 'The Syntax of Sudoku.pdf'; see module PsQ_GridCollection for creation of datasets and PsQ_Solver_CNN for model/training details; for a more advanced visualization of the Sudoku solver see GUI PsQ_Display. All can be found at https://github.com/A-Lex-McLee/PseudoQ-2.1/tree/main 

<br> PS: The dataset used here to train the model is relatively small/simple; while the model pretty much solves every puzzle constructed from the train/test dataset, it has problems with genuinely new data; check out PsQ_Display and try the option 'Grid_random'. 

Next Challenge: train a model that deals better with random/new puzzles!

  
<br>


McGuire, Gary, Bastian Tugemann, and Gilles Civario. 2014. There is no 16-clue Sudoku: Solving the Sudoku minimum number of clues problem via hitting set enumeration. *Experimental Mathematics* 23 (2): 190–217.


In [17]:
gc = GC.from_scratch()
idx = np.random.randint(gc.SIZE_collection)
gc.activate_horizontalSeries(idx)
gc.activate_puzzleSeries(64)
print()
print(gc)

data = gc.split_puzzle(train_ratio=0.81, seed = 17)
X_train, X_test, y_train, y_test = (arr.reshape(-1, 9, 9) for arr in data)
y_train, y_test = (y - 1 for y in (y_train, y_test))


Generating grids with 40-64 blanks: 100%|██████████| 1088640/1088640 [00:13<00:00, 79944.55it/s]



GridCollection[ 
                 internal shape: (1679616, 81)  
                 active series: k_64_max_puzzle_from_horizontal 
	                size: 2540160  
                 labeled series: 64 k_blanks  
                 labels: True  
	               size: 2540160  
                 garbage: False 
	               total size: 0 
					 guest_grids              : 0
					 false_fromCurrent_seq    : 0
					 false_fromCurrent_switch : 0
					 false_cardinality        : 0
					 false_off_by_X           : 0
					 false_arbitrary          : 0 
                 one_hot encoding: False  
                 internal abc collection: False  
               ] 


In [ ]:
solver = Grid_CNN_Solver(data)
solver.model.summary()
solver.fit()

solver.save_model(filename="solver1")
solver.save_data(filename="dataset_1")

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 9, 9)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_7 (Embedding)         │ (None, 9, 9, 16)       │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 9, 9, 81)       │        11,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 9, 9, 81)       │           324 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_21 (Activation)      │ (None, 9, 9, 81)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 9, 9, 135)      │        98,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 9, 9, 180)      │       218,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 9, 9, 180)      │           720 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_22 (Activation)      │ (None, 9, 9, 180)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 9, 9, 243)      │       393,903 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 9, 9, 9)        │         2,196 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_23 (Activation)      │ (None, 9, 9, 9)        │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,177,609 (8.31 MB)

 Trainable params: 725,695 (2.77 MB)

 Non-trainable params: 522 (2.04 KB)

 Optimizer params: 1,451,392 (5.54 MB)

In [19]:
## - evaluate model 
print("Evaluate test set: ")
solver.evaluate()
print()
print("Evaluate full dataset: ")
solver.evaluate_full() 

Evaluate test set: 
15083/15083 ━━━━━━━━━━━━━━━━━━━━ 239s 16ms/step - accuracy: 0.9865 - loss: 0.0307

Evaluate full dataset: 
79380/79380 ━━━━━━━━━━━━━━━━━━━━ 1284s 16ms/step - accuracy: 0.9874 - loss: 0.0288


In [21]:
# sanity check -- is the accuracy score really justified, the grids correctly predicted?
dat = solver.maskedGrids
lab = solver.labels
y_preds_full = solver.model.predict(dat)
full_p = np.argmax(y_preds_full, axis=-1)
matches               = np.all(full_p.reshape(-1, 81) == lab.reshape(-1, 81), axis=1)    
fully_identical_grids = np.sum(matches)     
matching_indices      = np.where(matches)[0]  

print(f"Number of total grid matches -- prediction == label:\n {fully_identical_grids} = {round(fully_identical_grids/lab.shape[0] * 100, 5)}%")


79380/79380 ━━━━━━━━━━━━━━━━━━━━ 1083s 14ms/step
Number of total grid matches -- prediction == label:
 1896561 = 74.66305%


In [ ]:
# practical application: let the AI solve a Sudoku puzzle 

# NB: if you don't run the above code, make sure you load model & dataset: 
# model = keras.models.load_model("solver1.keras")
# dat = np.load("reduced_dataset_1.npy")


sudoku = SudokuPlayer(model=solver.model, 
                        k_blanks=50, 
                        data=dat)        

sudoku.lets_play()

# press 'Enter' to fill in next cell

[[8 0 0 0 0 0 0 9 2]
 [2 0 0 0 7 0 0 1 6]
 [0 0 0 9 0 0 0 0 0]
 [5 0 8 4 0 0 0 0 0]
 [7 0 0 0 0 0 0 4 0]
 [4 0 6 0 3 0 1 0 5]
 [6 0 0 0 1 0 2 7 4]
 [0 0 2 0 8 0 9 5 1]
 [0 0 0 5 4 2 0 0 0]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Inserted: ((2, 4), 2)
next move: 

-------------------------
| 8 0 0 | 0 0 0 | 0 9 2 | 
| 2 0 0 | 0 7 0 | 0 1 6 | 
| 0 0 0 | 9 2 0 | 0 0 0 | 
-------------------------
| 5 0 8 | 4 0 0 | 0 0 0 | 
| 7 0 0 | 0 0 0 | 0 4 0 | 
| 4 0 6 | 0 3 0 | 1 0 5 | 
-------------------------
| 6 0 0 | 0 1 0 | 2 7 4 | 
| 0 0 2 | 0 8 0 | 9 5 1 | 
| 0 0 0 | 5 4 2 | 0 0 0 | 
-------------------------
True

[[8 0 0 0 0 0 0 9 2]
 [2 0 0 0 7 0 0 1 6]
 [0 0 0 9 2 0 0 0 0]
 [5 0 8 4 0 0 0 0 0]
 [7 0 0 0 0 0 0 4 0]
 [4 0 6 0 3 0 1 0 5]
 [6 0 0 0 1 0 2 7 4]
 [0 0 2 0 8 0 9 5 1]
 [0 0 0 5 4 2 0 0 0]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Inserted: ((2, 7), 8)
next move: 

-------------------------
| 8 0 0 | 0 0 0 | 0 9 2 | 
| 2 0 0 | 0 7 0 | 0 1 6 | 
| 0 0 0 | 9 2 0 | 0 8 0 | 
-------------